# Building a WiFi spots Static & Interactive Map of Networks around you with WiGLE and Python:

In [1]:
# IMPORTING REQUIRED LIBRARIES:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import geoplotlib as gp
%matplotlib notebook

# FOR INTERACTIVITY & OTHER OPTIONS WITH GEOPLOTLIB:
#from geoplotlib.layers import BaseLayer
#from geoplotlib.core import BatchPainter
#from geoplotlib.colors import colorbrewer
#from geoplotlib.utils import epoch_to_str, BoundingBox

In [8]:
# SETTING WiGLE USERNAME & PASSWORD FOR API CALL:
wigle_username = 'AID598df7f81d908d0b96d1'
wigle_password = '76343e08088e5b46d3454c0'

In [3]:
# SETTING PARAMETERS:
payload = {'addresscode': 'Pune', 'api_key': (wigle_username + wigle_password).encode()}

# FETCHING JSON RESPONSE FROM WiGLE:
pune_details = requests.get(url='https://api.wigle.net/api/v2/network/geocode', params=payload, 
                       auth=HTTPBasicAuth(wigle_username, wigle_password)).json()

# PRINTING TO CHECK BOUNDING BOX:
pune_details

{'success': True,
 'results': [{'address': {'city': 'Pune',
    'county': 'Pune',
    'state_district': 'Pune District',
    'state': 'Maharashtra',
    'postcode': '411001',
    'country': 'भारत',
    'country_code': 'in'},
   'lat': 18.5203062,
   'lon': 73.8543185,
   'importance': 0.69814202132834,
   'place_id': 110025,
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright',
   'osm_type': 'node',
   'display_name': 'Pune, Pune District, Maharashtra, 411001, भारत',
   'boundingbox': [18.3603062, 18.6803062, 73.6943185, 74.0143185]}]}

In [5]:
# SETTING PARAMETERS:
payload = {'latrange1':'18.3603062', 'latrange2':'18.6803062', 'longrange1':'73.6943185', 'longrange2':'74.0143185', 
           'api_key': (wigle_username + wigle_password).encode()}

# FETCHING JSON RESPONSE FROM WiGLE:
pune_results = requests.get(url='https://api.wigle.net/api/v2/network/search', params=payload, 
                       auth=HTTPBasicAuth(wigle_username, wigle_password)).json()

# PRINTING TO CHECK PUNE RESULTS (PULLING 1 OUT OF FETCHED 100):
pune_results['results'][0]

{'trilat': 18.50856781,
 'trilong': 73.93006134,
 'ssid': 'LIKLA',
 'qos': 0,
 'transid': '20140728-00000',
 'channel': 6,
 'firsttime': '2014-07-28T15:00:00.000Z',
 'lasttime': '2014-07-28T05:00:00.000Z',
 'lastupdt': '2014-07-28T03:00:00.000Z',
 'housenumber': '',
 'road': 'Mundhwa Road',
 'city': 'Pune',
 'region': 'Maharashtra',
 'country': 'IN',
 'netid': '00:1D:7E:A2:79:DF',
 'name': None,
 'type': 'infra',
 'comment': None,
 'wep': 'W',
 'bcninterval': 0,
 'freenet': '?',
 'dhcp': '?',
 'paynet': '?',
 'userfound': False,
 'encryption': 'wpa'}

In [7]:
# EXTRACTING 'RESULTS' AS A PANDAS DATAFRAME TO WORK WITH:
df = json_normalize(pune_results['results'])

# RENAMING COLUMNS FOR GEOPLOTLIB:
df = df.rename(columns={'trilat': 'lat', 'trilong': 'lon'})
cols = list(df.columns)

# PREVIEWING AVAILABE INFORMATION:
print(f"Result obtained has {df.shape[0]} rows and {df.shape[1]} columns in it. \n\nThe list of columns include {cols}.")
df.head(10)

Result obtained has 100 rows and 25 columns in it. 

The list of columns include ['bcninterval', 'channel', 'city', 'comment', 'country', 'dhcp', 'encryption', 'firsttime', 'freenet', 'housenumber', 'lasttime', 'lastupdt', 'name', 'netid', 'paynet', 'qos', 'region', 'road', 'ssid', 'transid', 'lat', 'lon', 'type', 'userfound', 'wep'].


,bcninterval,channel,city,comment,country,dhcp,encryption,firsttime,freenet,housenumber,...,qos,region,road,ssid,transid,lat,lon,type,userfound,wep
0,0,6,Pune,None,IN,?,wpa,2014-07-28T15:00:00.000Z,?,,...,0,Maharashtra,Mundhwa Road,LIKLA,20140728-00000,18.508568,73.930061,infra,False,W
1,0,11,,None,IN,?,wpa2,2016-11-15T06:00:00.000Z,?,,...,4,Maharashtra,Aditya Birla Hospital Marg,VirusLink,20161114-00000,18.606792,73.754868,infra,False,2
2,0,6,,None,IN,?,wpa2,2017-03-06T12:00:00.000Z,?,,...,0,Maharashtra,Sutarwadi Road,Negi,20170306-00000,18.536802,73.790268,infra,False,2
3,0,11,Pune,None,IN,?,wpa,2015-04-22T16:00:00.000Z,?,,...,0,Maharashtra,East Street,mynetsec,20150422-00000,18.513580,73.879684,infra,False,W
4,0,6,Pune,None,IN,?,wpa2,2014-07-26T16:00:00.000Z,?,,...,0,Maharashtra,Old Pune–Mumbai Highway,LTICONET,20140726-00000,18.534588,73.851532,infra,False,2
5,0,1,Pune,None,IN,?,wpa2,2014-07-26T16:00:00.000Z,?,,...,0,Maharashtra,Old Pune–Mumbai Highway,CORP-WIFI-2,20140726-00000,18.534742,73.851410,infra,False,2
6,0,11,Pune,None,IN,?,wpa,2014-09-10T15:00:00.000Z,?,,...,2,Maharashtra,Quarter Gate,Arjansingh,20140910-00000,18.517504,73.871170,infra,False,W
7,0,2,,None,IN,?,wpa2,2017-01-25T14:00:00.000Z,?,,...,0,Maharashtra,Bypass,dlink-321e,20170125-00000,18.542240,73.777328,infra,False,2
8,0,11,Pune,None,IN,?,wpa2,2016-12-05T15:00:00.000Z,?,,...,0,Maharashtra,Mohammadwadi Road,RaviWifi,20161205-00000,18.467628,73.920509,infra,False,2
9,0,5,Pune,None,IN,?,wpa2,2014-11-29T17:00:00.000Z,?,,...,0,Maharashtra,Mahatma Gandhi Marg,Aurora Tower,20141130-00000,18.518353,73.879616,infra,False,2


In [6]:
# GENERATING WIFI SPOTS MAP OF NETWORKS:
gp.dot(df)
gp.show()

<img src="D:\CRITICAL\New_Desktop\geo.png">

## Adding Interactivity using Folium:

In [7]:
# CREATING AN EMPTY NEW COLUMN FOR FOLIUM:
df['color'] = ''

# PICKING 'UNKNOWN' & 'NONE' VALUES FROM 'ENCRYPTION' COLUMN TO FEED 'COLOR' COLUMN:
df['color'] = df['encryption'].map({'unknown':'yellow', 'none':'red', 'wep':'blue', 'wpa':'blue', 'wpa2':'blue'})

# RECHECKING 'ENCRYPTION' & 'COLOR' DATAFRAME:
df[['encryption', 'color']].tail(10)

,encryption,color
90,wpa2,blue
91,wpa2,blue
92,unknown,yellow
93,wpa2,blue
94,wpa2,blue
95,unknown,yellow
96,unknown,yellow
97,none,red
98,wpa2,blue
99,wpa2,blue


In [8]:
# INTERACTIVE MAP WITH FOLIUM:
import folium

In [9]:
# SETTING BASE MAP FOR PUNE WITH A TILE/THEME:
pune_map = folium.Map([18.3603062, 74.0143185], zoom_start=6, tiles='cartodbdark_matter')

# LET US USE 'COLORS' COLUMN TO SEGREGATE 100 SSIDs 
# WITH CIRCLE MARKS AND ALSO ADD A POPUP MESSAGE:
df.apply(lambda row: folium.CircleMarker(location= [row['lat'], row['lon']], radius= 5, color= row['color'],
                                         popup= 'SSID: %s /n Encrpytion %s' %(row['ssid'], row['encryption']))
         .add_to(pune_map), axis=1)

# CHECKING OUTCOME & SAVING A COPY OF MAP:
pune_map.save('Pune Map')
pune_map